In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sbitraining/HACKATHON_TRAINING_DATA.CSV
/kaggle/input/prediction/HACKATHON_PREDICTION_DATA.CSV.xls


In [4]:
import pandas as pd

df = pd.read_csv('/kaggle/input/sbitraining/HACKATHON_TRAINING_DATA.CSV')
print(df.head())
print(df.columns)

   ACCT_AGE      LIMIT        OUTS  ACCT_RESIDUAL_TENURE  LOAN_TENURE  \
0     1.613  1005500.0   494161.89                 0.890          914   
1     1.783  1005500.0   428072.24                 0.720          914   
2     1.698  1005500.0   461364.10                 0.805          914   
3     9.127  1005500.0  1204287.25                17.878         9862   
4     9.296  1005500.0  1203224.25                17.708         9862   

   INSTALAMT SI_FLG     AGE  VINTAGE  KYC_SCR  ... CREDIT_HISTORY_LENGTH1  \
0    38513.0      Y  57.663   18.601    110.0  ...              7yrs 6mon   
1    38513.0      Y  57.833   18.771    110.0  ...              7yrs 6mon   
2    38513.0      Y  57.748   18.686    110.0  ...              7yrs 6mon   
3    12736.0      Y  52.302   14.039    110.0  ...             10yrs 8mon   
4    12736.0      Y  52.472   14.209    110.0  ...             10yrs 8mon   

  NO_OF_INQUIRIES1 INCOME_BAND1           AGREG_GROUP   PRODUCT_TYPE  \
0              0.0        

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix, recall_score
from sklearn.utils import class_weight
import xgboost as xgb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Recall

# ✅ Step 1: Preprocessing
X = df.drop(columns=['TARGET', 'UNIQUE_ID']).copy()
y = df['TARGET']

# Handle placeholders and encode categoricals
X.replace(["\\N", "NA", "NaN", "null", ""], np.nan, inplace=True)
for col in X.select_dtypes(include=['object', 'category']).columns:
    X[col] = X[col].astype(str)
    X[col] = LabelEncoder().fit_transform(X[col])

# Impute missing values
X = pd.DataFrame(SimpleImputer(strategy='median').fit_transform(X), columns=X.columns)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, test_size=0.3, random_state=42)
print(f"🔹 Train Rows: {len(X_train)}")
print(f"🔹 Test Rows:  {len(X_test)}")

2025-08-02 07:17:41.648341: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754119061.860074      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754119061.920680      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🔹 Train Rows: 229418
🔹 Test Rows:  98323


#### inference

In [6]:
# Load test.csv for inference (no TARGET column)
test_df = pd.read_csv('/kaggle/input/prediction/HACKATHON_PREDICTION_DATA.CSV.xls')
X_test_inference = test_df.drop(columns=['UNIQUE_ID']).copy()
test_unique_ids = test_df['UNIQUE_ID'].copy()

# Handle placeholders and encode categoricals
X_test_inference.replace(["\\N", "NA", "NaN", "null", ""], np.nan, inplace=True)
for col in X_test_inference.select_dtypes(include=['object', 'category']).columns:
    X_test_inference[col] = X_test_inference[col].astype(str)
    X_test_inference[col] = LabelEncoder().fit_transform(X_test_inference[col])

# Impute missing values
X_test_inference = pd.DataFrame(SimpleImputer(strategy='median').fit_transform(X_test_inference), columns=X_test_inference.columns)

# Scale features using the same scaler fitted on training data
X_test_scaled = scaler.transform(X_test_inference)  # Use transform, not fit_transform
print(f"🔹 Test inference rows: {len(X_test_scaled)}")

🔹 Test inference rows: 191693


In [7]:
import xgboost as xgb
from sklearn.metrics import recall_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np

# Assume X, y already preprocessed and split:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, test_size=0.3, random_state=42)

# Calculate scale_pos_weight: (negatives / positives)
neg, pos = np.bincount(y_train)
scale_pos_weight = neg / pos

# Initialize XGBoost classifier with scale_pos_weight to prioritize recall
model = xgb.XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    scale_pos_weight=scale_pos_weight,
    max_depth=6,
    learning_rate=0.1,
    n_estimators=100,
    objective='binary:logistic',
    subsample=0.8,
    colsample_bytree=0.8,
    seed=42
)

# Train model
model.fit(X_train, y_train)

# Predict probabilities
y_probs = model.predict_proba(X_test)[:, 1]

# Tune threshold to maximize recall
threshold = 0.3  # Lower threshold to increase recall
y_pred = (y_probs > threshold).astype(int)

# Evaluation
print(f"Recall at threshold {threshold}: {recall_score(y_test, y_pred):.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print(f"TP: {tp}, FP: {fp}, TN: {tn}, FN: {fn}")


Recall at threshold 0.3: 0.9316

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.70      0.82     87691
           1       0.28      0.93      0.43     10632

    accuracy                           0.73     98323
   macro avg       0.63      0.82      0.62     98323
weighted avg       0.91      0.73      0.78     98323

TP: 9905, FP: 25932, TN: 61759, FN: 727


#### inference

In [8]:
# Stage 1: XGBoost predictions with threshold 0.3
test_probs_stage1 = model.predict_proba(X_test_scaled)[:, 1]
test_pred_stage1 = (test_probs_stage1 > 0.3).astype(int)

print(f"Stage 1 Results:")
print(f"Total test samples: {len(test_pred_stage1)}")
print(f"Stage 1 positive predictions: {test_pred_stage1.sum()}")
print(f"Stage 1 negative predictions: {len(test_pred_stage1) - test_pred_stage1.sum()}")
print(f"Samples proceeding to Stage 2: {test_pred_stage1.sum()}")

Stage 1 Results:
Total test samples: 191693
Stage 1 positive predictions: 72272
Stage 1 negative predictions: 119421
Samples proceeding to Stage 2: 72272


In [9]:
# Attach predictions and probabilities to the original test set
original_test_df = df.loc[y_test.index].copy()
original_test_df['y_pred'] = y_pred
original_test_df['y_prob'] = y_probs


# ✅ df1: All predicted frauds (y_pred == 1)
df1 = original_test_df[original_test_df['y_pred'] == 1].copy()
df1_p1 = df1['y_prob'].values

# ✅ df2: All predicted non-frauds (y_pred == 0)
df2 = original_test_df[original_test_df['y_pred'] == 0].copy()
df2_p1 = df2['y_prob'].values


In [10]:
# Prepare data for Stage 2: only Stage 1 positives
stage1_positive_indices = np.where(test_pred_stage1 == 1)[0]

if len(stage1_positive_indices) > 0:
    # Create dataframe for Stage 2 processing
    test_stage2_df = test_df.iloc[stage1_positive_indices].copy()
    test_stage2_df['stage1_pred'] = 1
    test_stage2_df['stage1_prob'] = test_probs_stage1[stage1_positive_indices]
    
    print(f"Stage 2 processing {len(test_stage2_df)} samples that were positive in Stage 1")
else:
    print("No samples were positive in Stage 1 - all predictions will be 0")
    test_stage2_df = pd.DataFrame()

Stage 2 processing 72272 samples that were positive in Stage 1


In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


# Step 1: Split features and target
X = df1.drop(columns=['TARGET', 'UNIQUE_ID']).copy()
y = df1['TARGET']

# Step 2: Handle missing values and encode categoricals
X.replace(["\\N", "NA", "NaN", "null", ""], np.nan, inplace=True)
for col in X.select_dtypes(include=['object', 'category']).columns:
    X[col] = X[col].astype(str)
    X[col] = LabelEncoder().fit_transform(X[col])

# Step 3: Impute and scale
X = pd.DataFrame(SimpleImputer(strategy='median').fit_transform(X), columns=X.columns)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 4: Train/test split (for model training)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, test_size=0.3, random_state=42)

# Step 5: Define models
models = [
    ('XGBoost', XGBClassifier(use_label_encoder=False, eval_metric='logloss')),
    ('LightGBM', LGBMClassifier()),
    ('CatBoost', CatBoostClassifier(verbose=0)),
    ('ExtraTrees', ExtraTreesClassifier()),
    ('MLP', MLPClassifier(max_iter=300)),
    ('LogisticRegression', LogisticRegression(max_iter=500)),
    ('RandomForest', RandomForestClassifier())
]

# Step 6: Fit models and predict on full X_scaled
output_df = pd.DataFrame()
for i, (name, model) in enumerate(models, 1):
    print(f"⏳ Training {name}...")
    model.fit(X_train, y_train)
    probs = model.predict_proba(X_scaled)[:, 1]  # Probabilities on full Stage 2 data
    output_df[f'model_{i}_prob'] = probs

# Add true labels
output_df['TARGET'] = y.values

# Step 7: Save output
output_df.to_csv("stage2_prediction_matrix_with_target.csv", index=False)
print("✅ Saved stage2_prediction_matrix_with_target.csv")


⏳ Training XGBoost...
⏳ Training LightGBM...
[LightGBM] [Info] Number of positive: 6933, number of negative: 18152
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.020616 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26895
[LightGBM] [Info] Number of data points in the train set: 25085, number of used features: 138
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.276380 -> initscore=-0.962488
[LightGBM] [Info] Start training from score -0.962488
⏳ Training CatBoost...
⏳ Training ExtraTrees...
⏳ Training MLP...
⏳ Training LogisticRegression...
⏳ Training RandomForest...
✅ Saved stage2_prediction_matrix_with_target.csv


#### inference

In [13]:
# Stage 2 processing: ensemble models for Stage 1 positives only
if len(test_stage2_df) > 0:
    X_test_stage2 = test_stage2_df.drop(columns=['UNIQUE_ID']).copy()
    
    # Handle placeholders and encode categoricals
    X_test_stage2.replace(["\\N", "NA", "NaN", "null", ""], np.nan, inplace=True)
    for col in X_test_stage2.select_dtypes(include=['object', 'category']).columns:
        X_test_stage2[col] = X_test_stage2[col].astype(str)
        X_test_stage2[col] = LabelEncoder().fit_transform(X_test_stage2[col])
    
    # Impute missing values
    X_test_stage2 = pd.DataFrame(SimpleImputer(strategy='median').fit_transform(X_test_stage2), columns=X_test_stage2.columns)
    
    # Scale features
    scaler_stage2 = StandardScaler()
    X_test_stage2_scaled = scaler_stage2.fit_transform(X_test_stage2)
    
    # Generate ensemble predictions
    test_ensemble_df = pd.DataFrame()
    for i, (name, model_obj) in enumerate(models, 1):
        probs = model_obj.predict_proba(X_test_stage2_scaled)[:, 1]
        test_ensemble_df[f'model_{i}_prob'] = probs
    
    print(f"Generated ensemble predictions for {len(test_ensemble_df)} Stage 2 samples")
else:
    test_ensemble_df = pd.DataFrame()

Generated ensemble predictions for 72272 Stage 2 samples


In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score

# Load the prediction matrix
df = output_df

# Feature matrix (7 model probs)
X = df.drop(columns=['TARGET'])
y = df['TARGET']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)


In [15]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Load prediction matrix
df = pd.read_csv("stage2_prediction_matrix_with_target.csv")

# Features and label
X = df.drop(columns=['TARGET'])
y = df['TARGET']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Train a model (you can replace this with any classifier)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predict probabilities
y_probs = model.predict_proba(X_test)[:, 1]

# === 🔥 TUNE THIS THRESHOLD ===
threshold = 0.25

# Apply threshold
y_pred = (y_probs >= threshold).astype(int)

# Confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

# Print breakdown
print("=== Confusion Matrix Breakdown ===")
print(f"Actual 0 → Predicted 0: {tn}")
print(f"Actual 0 → Predicted 1: {fp}")
print(f"Actual 1 → Predicted 0: {fn}")
print(f"Actual 1 → Predicted 1: {tp}")
print()

# Metrics
precision = tp / (tp + fp) if (tp + fp) != 0 else 0
recall    = tp / (tp + fn) if (tp + fn) != 0 else 0
f1_score  = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
auc       = roc_auc_score(y_test, y_probs)

print("=== Metrics at Threshold =", threshold, "===")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1_score:.4f}")
print(f"AUC      : {auc:.4f}")


=== Confusion Matrix Breakdown ===
Actual 0 → Predicted 0: 4146
Actual 0 → Predicted 1: 1041
Actual 1 → Predicted 0: 720
Actual 1 → Predicted 1: 1261

=== Metrics at Threshold = 0.25 ===
Precision: 0.5478
Recall   : 0.6365
F1 Score : 0.5888
AUC      : 0.7919


#### inference

In [17]:
# Stage 2 final predictions using trained LogisticRegression model
if len(test_ensemble_df) > 0:
    test_probs_stage2 = model.predict_proba(test_ensemble_df)[:, 1]  # Use trained LogisticRegression
    test_pred_stage2 = (test_probs_stage2 >= 0.25).astype(int)  # Threshold 0.05
    
    print(f"Stage 2 Results:")
    print(f"Stage 2 positive predictions: {test_pred_stage2.sum()}")
    print(f"Stage 2 negative predictions: {len(test_pred_stage2) - test_pred_stage2.sum()}")
else:
    test_pred_stage2 = np.array([])
    print("No Stage 2 predictions needed")

Stage 2 Results:
Stage 2 positive predictions: 21092
Stage 2 negative predictions: 51180


# INFERENCE

In [18]:
# Final Two-Stage Predictions for Test Dataset
# Stage 1: All samples → XGBoost with threshold 0.3
# Stage 2: Only Stage 1 positives → Ensemble + LogisticRegression with threshold 0.05

# Initialize all predictions as 0
final_test_predictions = np.zeros(len(test_df), dtype=int)

print(f"=== Two-Stage Prediction Pipeline ===")
print(f"Total test samples: {len(test_df)}")
print(f"Stage 1 positives: {test_pred_stage1.sum()}")

# Only Stage 1 positives that also pass Stage 2 become final positives
if len(test_pred_stage2) > 0:
    final_test_predictions[stage1_positive_indices] = test_pred_stage2
    final_positive_count = final_test_predictions.sum()
    
    print(f"Stage 2 positives: {test_pred_stage2.sum()}")
    print(f"Final positives: {final_positive_count}")
else:
    final_positive_count = 0
    print(f"Stage 2 positives: 0")
    print(f"Final positives: 0")

print(f"Final negatives: {len(final_test_predictions) - final_positive_count}")
print(f"Final fraud detection rate: {final_positive_count/len(final_test_predictions)*100:.2f}%")

# Create final output CSV
final_test_output = pd.DataFrame({
    'UNIQUE_ID': test_unique_ids,
    'FINAL_PREDICTION': final_test_predictions
})

# Save predictions
final_test_output.to_csv('test_predictions.csv', index=False)
print(f"\n✅ Final test predictions saved to 'test_predictions.csv'")
print(f"Output format: UNIQUE_ID, FINAL_PREDICTION")
print(f"Shape: {final_test_output.shape}")

# Show sample predictions
print(f"\nFirst 10 predictions:")
print(final_test_output.head(10))

=== Two-Stage Prediction Pipeline ===
Total test samples: 191693
Stage 1 positives: 72272
Stage 2 positives: 21092
Final positives: 21092
Final negatives: 170601
Final fraud detection rate: 11.00%

✅ Final test predictions saved to 'test_predictions.csv'
Output format: UNIQUE_ID, FINAL_PREDICTION
Shape: (191693, 2)

First 10 predictions:
   UNIQUE_ID  FINAL_PREDICTION
0       2202                 0
1       2209                 0
2       2211                 0
3       2217                 0
4       2218                 0
5       4197                 0
6       4198                 0
7       4200                 0
8       4202                 0
9       4203                 0
